# Data Collection

In [9]:
import requests
from bs4 import BeautifulSoup
import csv

In [2]:
import requests
from bs4 import BeautifulSoup

def scrape_titles():
    base_url = "https://community.ricksteves.com/travel-forum/turkey?page="
    page_number = 1
    all_titles = []

    while True:
        # Construct the URL for the current page
        url = f"{base_url}{page_number}"
        
        # Send a GET request to the URL
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code != 200:
            print(f"Failed to retrieve page {page_number}. Status code: {response.status_code}")
            break

        # Parse the HTML content of the page
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find all <tr> elements with the specified class
        rows = soup.find_all('tr', class_='topic-summary-data topic-summary')

        # If no rows are found, assume we've reached the last page
        if not rows:
            print(f"No more rows found on page {page_number}. Stopping.")
            break

        # Extract titles from the rows
        for row in rows:
            title_cell = row.find('td', class_='topic-summary-data title')
            if title_cell:
                link = title_cell.find('a')
                if link:
                    all_titles.append([link.text.strip(), page_number])

        print(f"Scraped page {page_number} with {len(rows)} titles.")
        page_number += 1

    return all_titles

# Call the function to scrape titles across all pages
titles = scrape_titles()

# Print the total number of titles scraped
print(f"Total titles scraped: {len(titles)}")


Scraped page 1 with 50 titles.
Scraped page 2 with 50 titles.
Scraped page 3 with 50 titles.
Scraped page 4 with 50 titles.
Scraped page 5 with 50 titles.
Scraped page 6 with 50 titles.
Scraped page 7 with 50 titles.
Scraped page 8 with 50 titles.
Scraped page 9 with 50 titles.
Scraped page 10 with 50 titles.
Scraped page 11 with 50 titles.
Scraped page 12 with 50 titles.
Scraped page 13 with 50 titles.
Scraped page 14 with 50 titles.
Scraped page 15 with 50 titles.
Scraped page 16 with 50 titles.
Scraped page 17 with 50 titles.
Scraped page 18 with 50 titles.
Scraped page 19 with 50 titles.
Scraped page 20 with 50 titles.
Scraped page 21 with 50 titles.
Scraped page 22 with 50 titles.
Scraped page 23 with 50 titles.
Scraped page 24 with 50 titles.
Scraped page 25 with 50 titles.
Scraped page 26 with 50 titles.
Scraped page 27 with 8 titles.
No more rows found on page 28. Stopping.
Total titles scraped: 1308


In [3]:
# save the titles to a csv to every row
with open('titles.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Title", "Page"])
    writer.writerows(titles)
print("Titles saved to 'titles.csv'")


Titles saved to 'titles.csv'


In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv

def follow_links_by_title(scraped_titles):
    visited_titles = set()
    qa_pairs = []  # List to store Q&A pairs

    # Set up the Selenium WebDriver (make sure you have the appropriate driver for your browser)
    driver = webdriver.Chrome()  # You can replace 'Chrome' with your preferred browser

    try:
        for title, page in scraped_titles:
            if title in visited_titles:
                continue

            try:
                # Navigate to the specific page
                page_url = f"https://community.ricksteves.com/travel-forum/turkey?page={page}"
                driver.get(page_url)

                # Find the link with the matching title
                link = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.LINK_TEXT, title))
                )

                # Click the link to navigate to the page
                link.click()

                # Scrape the content of the page
                qa_pairs.append(follow_and_scrape_page(driver, title))

                # Mark the title as visited
                visited_titles.add(title)

            except Exception as e:
                print(f"Failed to find or scrape the link for title: {title} on page {page}. Error: {e}")

    finally:
        # Close the browser
        driver.quit()

    return qa_pairs

def follow_and_scrape_page(driver, title):
    qa_dict = {}

    try:
        # Extract the main question
        question = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'div.content.markdown[itemprop="articleBody"]'))
        )
        qa_dict["question"] = question.text.strip()
    except Exception as e:
        print(f"Failed to extract the main question for title: {title}. Error: {e}")
        qa_dict["question"] = ""

    # Extract all replies
    try:
        replies = driver.find_elements(By.CSS_SELECTOR, 'article.reply[itemprop="comment"]')
        for idx, reply in enumerate(replies, start=1):
            reply_text = reply.find_element(By.CSS_SELECTOR, 'div.content.markdown[itemprop="text"]').text.strip()
            qa_dict[f"answer{idx}"] = reply_text
    except Exception as e:
        print(f"Failed to extract replies for title: {title}. Error: {e}")

    # Add title to the dictionary
    qa_dict["title"] = title

    return qa_dict

# Read titles and pages from the CSV file
titles = []
with open('titles.csv', 'r') as file:
    reader = csv.reader(file)
    next(reader)  # Skip the header row
    for row in reader:
        titles.append((row[0], row[1]))  # Append title and page number as a tuple

# Scrape the Q&A pairs
qa_pairs = follow_links_by_title(titles)


Failed to find or scrape the link for title: What neighborhood do you love in Istanbul to stay?  Please! September... on page 4. Error: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7296BFB05+28789]
	(No symbol) [0x00007FF7296286E0]
	(No symbol) [0x00007FF7294C592A]
	(No symbol) [0x00007FF72951930E]
	(No symbol) [0x00007FF7295195FC]
	(No symbol) [0x00007FF7295628A7]
	(No symbol) [0x00007FF72953F47F]
	(No symbol) [0x00007FF72955F654]
	(No symbol) [0x00007FF72953F1E3]
	(No symbol) [0x00007FF72950A938]
	(No symbol) [0x00007FF72950BAA1]
	GetHandleVerifier [0x00007FF7299F933D+3410093]
	GetHandleVerifier [0x00007FF729A0E7DD+3497293]
	GetHandleVerifier [0x00007FF729A02A73+3448803]
	GetHandleVerifier [0x00007FF729787BBB+848171]
	(No symbol) [0x00007FF729633C3F]
	(No symbol) [0x00007FF72962F6E4]
	(No symbol) [0x00007FF72962F87D]
	(No symbol) [0x00007FF72961ED49]
	BaseThreadInitThunk [0x00007FFBF42C7374+20]
	RtlUserThreadStart [0x00007FFBF543CC91+33]

Failed to find or scrape the link for t

In [5]:
print(f"Scraped {len(qa_pairs)} Q&A pairs.")

Scraped 1254 Q&A pairs.


In [7]:
# Save the Q&A pairs to a CSV file
with open('qa_pairs.csv', 'w', newline='') as file:
    # write line by line
    writer = csv.writer(file)
    writer.writerow(["QA_pair"])
    for qa_pair in qa_pairs:
        writer.writerow([qa_pair])
print("Q&A pairs saved to 'qa_pairs.csv'")

Q&A pairs saved to 'qa_pairs.csv'


In [23]:
import requests
from groq import Groq
# API key for authentication
#api_key = "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"
api_key = "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"
#api_key = "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"

client = Groq(
    api_key=api_key,
)

# read the qa_pairs from the csv file
qa_pairs = []
with open('qa_pairs.csv', 'r') as file:
    reader = csv.reader(file)
    next(reader)  # Skip the header row
    for row in reader:
        qa_pairs.append(row[0])  # Append title and page number as a tuple

#qa_pairs = qa_pairs[:250]
#qa_pairs = qa_pairs[250:500]
#qa_pairs = qa_pairs[500:750]
qa_pairs = qa_pairs[750:1000]
#qa_pairs = qa_pairs[1000:]

istanbul_yes = list()
i = 0
for QA in qa_pairs:
    if i % 50 == 0:
        print(f"Checking title {i}...")
    i += 1
    # Construct the URL for the API request
    question = "Is this information is about İstanbul?, answer like just yes or no (lowercase), information=" + str(QA)

    chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": question,
        }
    ],
    model="llama3-8b-8192",)

    if str(chat_completion.choices[0].message.content) == "yes":

        istanbul_yes.append(QA)

print(f"Total number of titles about Istanbul: {len(istanbul_yes)} out of {len(qa_pairs)}")


Checking title 0...


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01jgkh89rxfr69n4fax56y5mv6` on : Limit 500000, Used 499979, Requested 418. Please try again in 1m8.529399999s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}

In [21]:
# save the titles to a csv
with open('istanbul_yes4.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    for title in istanbul_yes:
        writer.writerow([title])

In [ ]:

QA_llm = list()

for QA in istanbul_yes:

    question = QA["question"]
    # assign answers to the question ( every element in the dict that are not question and title)
    answers = [v for k, v in QA.items() if k.startswith("answer")]

    query = "Analyze this question: " + str(question) + " and answers: " + str(answers) + "and return me a simplified question and answer pair based on the answers provided. Make it short and brief. Return me a question and answer text in json format. DO NOT INCLUDE ANY OTHER THING! Return only in .json with the format of {\"question\": \"your question\", \"answer\": \"your answer\"}. do npt include any other thing in the response. "

    chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": query,
        }
    ],
    model="llama3-8b-8192",)

    QA_llm.append(chat_completion.choices[0].message.content)
    # Extract the answer from the response

# save the titles to a csv
with open('QA_llm.json', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(QA_llm)

print("All done!")

{"question": "Can I request a stopover with Turkish Airlines and which method did the OP use?", "answer": "Yes, Turkish Airlines allows stopovers, but the OP had to email the airline due to peculiarities in their reservation."}
{"question": "Has anyone tried the Turkish baths/Hammans in Istanbul? If so, would you recommend it? Which one in particular is good?", "answer": "Yes, many people have tried them and recommended it. Hurrem Sultan Hammami and Cagloglu are good options, but be prepared for it to be pricey."}
{"question": "What is the best way to get from Istanbul airport to the city center via taxi?", "answer": "Consider taking the metro (25 Turkish Lira) to Gayrettepe metro station, then take a taxi from there to pay less for the taxi."}
Here is the simplified question and answer pair in JSON format:

{"question": "What's your experience with the hotel provided in the 'Stopover in Istanbul' offer?", "answer": "We stayed at the Galata Hotel in the new district. In my opinion, it 